# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /Users/guyzidkia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/guyzidkia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/guyzidkia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Data_clean.db')
df = pd.read_sql ('SELECT * FROM Data_clean', engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
#for column in Y.columns:
 #   print (Y[column].unique())

In [4]:
Y.groupby("related").count()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,188,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [5]:
Y[Y["related"]==2] =1

In [6]:
Y.groupby("related").count()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094,...,20094,20094,20094,20094,20094,20094,20094,20094,20094,20094


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    tokens = word_tokenize(text)
    #print(text)
    #print(tokens)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    #print(clean_tokens)
    return clean_tokens
    pass


In [8]:
for sentence in X:
    tokenize(sentence)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [12]:
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame (y_pred, columns = y_test.columns)
print(classification_report(y_test.values, y_pred, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      4996
               request       0.84      0.37      0.51      1183
                 offer       0.50      0.03      0.06        67
           aid_related       0.77      0.45      0.57      2762
          medical_help       0.58      0.02      0.04       574
      medical_products       0.58      0.03      0.06       378
     search_and_rescue       0.50      0.02      0.05       201
              security       0.50      0.01      0.03       151
              military       0.50      0.01      0.03       224
           child_alone       0.50      0.06      0.11        34
                 water       0.86      0.15      0.26       460
                  food       0.84      0.30      0.44       780
               shelter       0.84      0.15      0.26       633
              clothing       0.33      0.01      0.03       151
                 money       0.75      

/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [13]:
y_pred.shape

(6554, 36)

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
#y_test.dtypes

In [15]:
#y_pred.dtypes

In [16]:
parameters = {
    
         'clf__n_estimators': [5, 10, 20],
        'clf__min_samples_split': [3, 4]
    }

cv = GridSearchCV (pipeline, param_grid= parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
model = cv
model.fit(X_train, y_train)
y_pred2 = model.predict (X_test)
#converting to a dataframe
y_pred2 = pd.DataFrame (y_pred2, columns = y_test.columns)

/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [18]:
print(classification_report(y_test.values, y_pred2, target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4996
               request       0.86      0.43      0.57      1183
                 offer       0.67      0.03      0.06        67
           aid_related       0.79      0.53      0.63      2762
          medical_help       0.71      0.03      0.05       574
      medical_products       0.50      0.02      0.04       378
     search_and_rescue       0.75      0.03      0.06       201
              security       0.67      0.01      0.03       151
              military       0.80      0.02      0.03       224
           child_alone       0.67      0.06      0.11        34
                 water       0.88      0.18      0.30       460
                  food       0.84      0.34      0.49       780
               shelter       0.85      0.13      0.22       633
              clothing       0.50      0.04      0.07       151
                 money       0.86      

/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [26]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

parameters = {
    
         'clf__n_estimators': [5, 10, 20],
        'clf__min_samples_split': [3, 4]
    }


In [27]:
cv = GridSearchCV (pipeline, param_grid = parameters)

In [ ]:
model = cv
model.fit(X_train, y_train)

/Users/guyzidkia/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
SVC().get_params().keys()

In [ ]:
y_pred = model.predict (X_test)
#converting to a dataframe
y_pred = pd.DataFrame (y_pred, columns = y_test.columns)

In [ ]:
reuslts = (classification_report(y_test.values, y_pred, target_names=y_test.columns.values))
print (reuslts3)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model2, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.